In [2]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2
import asyncio

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time

from qubx.impl.ccxt_connector import CCXTExchangesConnector
from qubx.impl.ccxt_trading import CCXTTradingConnector
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        
2024-07-19 00:11:09.013 [ ℹ️ ] NTP offset controller thread is started


In [3]:
class TestStrategy(IStrategy):
    capital_invested: int = 1000

    def on_start(self, ctx: StrategyContext):
        logger.info(f" -> Strategy started | {ctx.time()}")

    def on_fit(self, ctx: 'StrategyContext', fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f" -> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        logger.info(f" ||||| {event.time} -> {event} |||||")

        for i in ctx.instruments:
            logger.info(f"\t{i.symbol} : {ctx.broker_provider.get_quote(i.symbol)}")
        logger.info(f"- - - - - - - - - - - - - - - - - - - - - - -")
        return None

    def ohlcs(self, timeframe:str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def on_stop(self, ctx: StrategyContext):
        logger.info(f" -> strategy is stopped")

    def tracker(self, ctx: StrategyContext) -> PortfolioRebalancerTracker:
        return PortfolioRebalancerTracker(ctx, 1000, 0)

# Simulated broker and paper trading

In [3]:
from qubx.backtester.simulator import SimulatedTrading

In [4]:
auth = get_account_config('binance-qbx', './configs/.env')

sim = SimulatedTrading('binance', 10000, 'vip0_usdt', 'USDT')
exch = CCXTExchangesConnector('binance', sim, **auth)

instruments = [lookup.find_symbol('BINANCE', 'BTCUSDT')]
ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    exch, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=0),
    trigger_spec='1Min -5Sec',
)
S = ctx.strategy

2024-07-16 11:27:39.515 [ ℹ️ ] binance initialized - current time 1970-01-01T00:00:00.000000000
2024-07-16 11:27:39.524 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-16 11:27:39.524 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-16 11:27:39.525 [ 🐞 ] Triggering strategy on every 1Min bar after 55S
2024-07-16 11:27:39.524 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-16 11:27:39.524 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-16 11:27:39.525 [ 🐞 ] Triggering strategy on every 1Min bar after 55S


In [5]:
ctx.start()

2024-07-16 11:27:42.734 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT'] 
2024-07-16 11:27:42.734 [ ℹ️ ] Subscribed on ohlc updates for 1 symbols: 
		['BTCUSDT']
2024-07-16 11:27:42.734 [ ℹ️ ]  -> Strategy started | 1970-01-01T00:00:00.000000000
2024-07-16 11:27:42.734 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-16 11:27:42.734 [ ℹ️ ] (StrategyContext) strategy is started


2024-07-16 11:27:47.303 [ 🐞 ]  (2024-07-16T08:27:46.952262400) DATA : ('BTCUSDT', 'bar', {o:62781.990000 | h:62782.140000 | l:62652.000000 | c:62652.000000 | v:1037665.532096})
2024-07-16 11:27:47.312 [ 🐞 ] [2024-07-16T08:27:46.952262400]: Invoking TestStrategy on_fit('1970-01-01 00:00:00', 'None')
2024-07-16 11:27:47.313 [ ℹ️ ]  -> Fit is called | fit_time: 1970-01-01 00:00:00 / prev: None
2024-07-16 11:27:47.313 [ 🐞 ] [2024-07-16T08:27:46.952262400]: TestStrategy is fitted
2024-07-16 11:27:49.386 [ 🐞 ]  (2024-07-16T08:27:49.035166976) DATA : ('BTCUSDT', 'bar', {o:62781.990000 | h:62782.140000 | l:62651.990000 | c:62651.990000 | v:1041744.803504})
2024-07-16 11:27:51.462 [ 🐞 ]  (2024-07-16T08:27:51.111800320) DATA : ('BTCUSDT', 'bar', {o:62781.990000 | h:62782.140000 | l:62651.990000 | c:62655.990000 | v:1083726.049126})
2024-07-16 11:27:53.603 [ 🐞 ]  (2024-07-16T08:27:53.252714240) DATA : ('BTCUSDT', 'bar', {o:62781.990000 | h:62782.140000 | l:62651.990000 | c:62659.990000 | v:109945

In [ ]:
exch.get_historical_ohlcs('BTCUSDT', '5Min', 200)

In [15]:
print(exch.trading_service.get_position(instruments[0]))
print(exch.trading_service.get_orders())

??? [BINANCE:BTCUSDT]       0.00000      0.00    +0.0000 | nan        0.00
[]


In [11]:
ctx.trade('BTCUSDT', 0.1)

2024-07-13 16:27:46.163 [ ℹ️ ] (StrategyContext) sending market buy for 0.1 of BTCUSDT ...
2024-07-13 16:27:46.164 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100001 MARKET BUY 0.1 executed at 58708.005
2024-07-13 16:27:46.165 [ ℹ️ ]   ::  traded 0.1 for BTCUSDT @ 58708.005 -> 0.00
2024-07-13 16:27:46.165 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100001 MARKET BUY 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100001', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-13T13:27:45.682696960'), quantity=0.1, price=0, side='BUY', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_1', cost=0.0)

In [17]:
ctx.trade('BTCUSDT', -0.1)

2024-07-13 16:29:50.614 [ ℹ️ ] (StrategyContext) sending market sell for 0.1 of BTCUSDT ...
2024-07-13 16:29:50.622 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100001 MARKET SELL 0.1 executed at 58737.985
2024-07-13 16:29:50.622 [ ℹ️ ]   ::  traded -0.1 for BTCUSDT @ 58737.985 -> 0.00
2024-07-13 16:29:50.623 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100001 MARKET SELL 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100001', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-13T13:29:49.821481984'), quantity=0.1, price=0, side='SELL', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_1', cost=0.0)

In [10]:
ctx.trade('BTCUSDT', 0.1, 45000)

2024-07-13 16:26:15.073 [ ℹ️ ] (StrategyContext) sending limit buy for 0.1 of BTCUSDT ...
2024-07-13 16:26:15.074 [ 🐞 ] [OMS] BTCUSDT - registered SIM-ORDER-BTCUSDT-100003 LIMIT BUY 0.1 45000


Order(id='SIM-ORDER-BTCUSDT-100003', type='LIMIT', symbol='BTCUSDT', time=numpy.datetime64('2024-07-13T13:26:14.253228800'), quantity=0.1, price=45000, side='BUY', status='OPEN', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_3', cost=0.0)

In [12]:
ctx.cancel('BTCUSDT')

2024-07-13 16:26:23.040 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100003 LIMIT BUY 0.1 canceled
2024-07-13 16:26:23.041 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100003 LIMIT BUY 0.1 of BTCUSDT -> CANCELED
2024-07-13 16:26:23.041 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100003 LIMIT BUY 0.1 of BTCUSDT -> CANCELED


In [7]:
ctx.stop()
exch.close()

2024-07-16 11:28:12.310 [ ℹ️ ]  -> strategy is stopped
2024-07-16 11:28:12.310 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000245 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0000363 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	StrategyContext.handler took 0.0000044 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	strategy.on_fit took 0.0013473 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	StrategyContext._process_incoming_data took 1.4273392 secs
2024-07-16 11:28:12.317 [ ℹ️ ] 	loggers.notify took 0.0000015 secs
2024-07-16 11:28:12.317 [ ℹ️ ] 	strategy.on_event took 0.0006303 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000245 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0000363 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	StrategyContext.handler took 0.0000044 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	strategy.on_fit took 0.0013473 secs
2024-07-16 11:28:12.310 [ ℹ️ ] 	StrategyContext._process_incoming_data took 1.42733

# Simulated data provider

In [4]:
# r = CsvStorageDataReader("../tests/data/csv")
r = MultiQdbConnector('xlydian-data')

2024-07-19 00:11:11.977 [ 🐞 ] Connected to QuestDB at xlydian-data:8812


In [12]:
from qubx.backtester.simulator import SimulatedTrading, SimulatedExchange

In [13]:
sbroker = SimulatedTrading('binance', 10000, 'vip0_usdt', 'USDT', '2024-01-01 00:00:00')
sdp = SimulatedExchange('binance', sbroker, r)

instruments = [lookup.find_symbol('BINANCE.UM', s) for s in ['BTCUSDT', 'SOLUSDT', 'LINKUSDT']]

ctx = StrategyContext(
    TestStrategy(), dict(capital_invested=100, trading_allowed=False), 
    sdp, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=0),
    # trigger_spec='5Min -1Sec',
    trigger_spec="*/5 * * * *"

)
ctx.start()

2024-07-19 00:12:28.802 [ ℹ️ ] SimulatedData.binance initialized
2024-07-19 00:12:28.803 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-19 00:12:28.804 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-19 00:12:28.805 [ 🐞 ] Now is 2024-01-01 00:00:00 next (time_event) at 2024-01-01 00:05:00
2024-07-19 00:12:28.806 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT', 'SOLUSDT', 'LINKUSDT'] 
2024-07-19 00:12:28.807 [ ℹ️ ]  -> Strategy started | 2024-01-01T00:00:00.000000000
2024-07-19 00:12:28.807 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-19 00:12:28.809 [ ℹ️ ] (StrategyContext) strategy is started
2024-07-19 00:12:28.803 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-19 00:12:28.804 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capit

In [14]:
# %%time
d = sdp.run('2024-01-01 00:00', '2024-01-01 00:25')

2024-07-19 00:12:29.670 [ 🐞 ] [2024-01-01T00:00:00.001000000]: Invoking TestStrategy on_fit('2024-01-01 00:00:00', 'None')
2024-07-19 00:12:29.671 [ ℹ️ ]  -> Fit is called | fit_time: 2024-01-01 00:00:00 / prev: None
2024-07-19 00:12:29.672 [ 🐞 ] [2024-01-01T00:00:00.001000000]: TestStrategy is fitted
2024-07-19 00:12:29.678 [ ℹ️ ]  ||||| 2024-01-01T00:05:00.001000000 -> TriggerEvent(time=numpy.datetime64('2024-01-01T00:05:00.001000000'), type='time', instrument=None, data=(1704066900.0, 1704067500.0)) |||||
2024-07-19 00:12:29.679 [ ℹ️ ] 	BTCUSDT : [2024-01-01T00:05:00.001000000]	42437.15000 (1000000000.0) | 42437.25000 (1000000000.0)
2024-07-19 00:12:29.679 [ ℹ️ ] 	SOLUSDT : [2024-01-01T00:05:00.001000000]	102.35550 (1000000000.0) | 102.35650 (1000000000.0)
2024-07-19 00:12:29.679 [ ℹ️ ] 	LINKUSDT : [2024-01-01T00:05:00.001000000]	15.02050 (1000000000.0) | 15.02150 (1000000000.0)
2024-07-19 00:12:29.680 [ ℹ️ ] - - - - - - - - - - - - - - - - - - - - - - -
2024-07-19 00:12:29.678 [ ℹ️

2024-07-19 00:12:29.678 [ 🐞 ] Now is 2024-01-01 00:05:00.000999928 next (time_event) at 2024-01-01 00:10:00
2024-07-19 00:12:29.684 [ 🐞 ] Now is 2024-01-01 00:10:00.000999928 next (time_event) at 2024-01-01 00:15:00
2024-07-19 00:12:29.684 [ 🐞 ] Now is 2024-01-01 00:10:00.000999928 next (time_event) at 2024-01-01 00:15:00
2024-07-19 00:12:29.690 [ 🐞 ] Now is 2024-01-01 00:15:00.000999928 next (time_event) at 2024-01-01 00:20:00
2024-07-19 00:12:29.696 [ 🐞 ] Now is 2024-01-01 00:20:00.000999928 next (time_event) at 2024-01-01 00:25:00
2024-07-19 00:12:29.702 [ 🐞 ] Now is 2024-01-01 00:25:00.000999928 next (time_event) at 2024-01-01 00:30:00


In [ ]:
ctx.stop()

In [ ]:
str(sbroker.get_position(instruments[0]))

1721316593.471884

In [74]:
from croniter import croniter
_SEC2TS = lambda t: pd.Timestamp(t, unit="s")

t0 = pd.Timestamp('2024-01-01 00:00:00').timestamp()

iter = croniter("*/5 * * * *", t0)

In [75]:
_SEC2TS(iter.get_next())

Timestamp('2024-01-01 00:05:00')